In [6]:
import re
import yaml

import sys
sys.path.append('/Users/xmkqv/projects/diff-ts')

from things.core import run_cmds, str2yaml, lo_ve

In [12]:
class Kubernetes():

	# create, get, describe, delete
	last_pod_created = None
	n_gpu 			= 1
	namespace 		= 'scb-usra'
	pod_name 		= 'gpu-pod-xmkqv'
	container_name 	= 'gpu-container'
	mount_path 		= '/scb-usra'
	link_cmd 		= 'ln -s /cephfs ~/cephfs'
	kub_image 		= 'gitlab-registry.nrp-nautilus.io/prp/jupyter-stack/prp:latest'
	get_pods 		= property(lambda _: f'kubectl get pods -n {_.namespace}')
	get_config 		= property(lambda _: f'kubectl config view')
	repo 			= 'github.com:xmax1/diff-ts.git'

	kub_c = property(lambda _: {
		'apiVersion': 'v1',
		'kind': 'Pod',
		'metadata': {'name': _.pod_name, 'namespace': _.namespace},
		'spec': {
			'containers': [
				{
				'name': _.container_name,
				'image': _.kub_image,
				'resources': {'limits': {'nvidia.com/gpu': _.n_gpu} },

				'command': ["/bin/bash","-c"], 
				'args': ['pip install tqdm; sleep infinity'],
				# 'args': ['conda env update --file env.yml; sleep infinity'],
				
				'volumeMounts': [
					{'mountPath': _.mount_path, 'name': _.mount_path.split('/')[-1]},
				]
				}
			],

			'volumes': [
				{'name': 'scb-usra', 'persistentVolumeClaim': {'claimName': 'scb-usra'}},
			],
			'restartPolicy': 'Never',
		}, # spec
		
	})
	
	def enter_pod(ii, pod_yaml: str= None):
		pod_yaml = pod_yaml or ii.last_pod_created
		"""-i, --stdin=false:
			Pass stdin to the container

		-t, --tty=false:
			Stdin is a TTY"""
		run_cmds(f'kubectl exec -f {pod_yaml} -n {ii.namespace} -it -- /bin/bash')

		
	
	def cluster_submit(ii, job: dict= None, kub_c: str= None):
		kub_c = kub_c or ii.kub_c
		path = ii.p.paths.exp_dir/'kub_c.yaml'
		
		c = ii.str2yaml(kub_c)
		with open(path, 'w') as f:
			f.write(c)

		print('Creating pod yaml ', path)
		run_cmds(f'kubectl create -f {path} -o yaml', silent= False)
		ii.last_pod_created = path
	
	def pr(ii, data: dict= None):
		import pprint
		c = data or ii.kub_c
		pprint.pprint(c)

In [13]:
kub = Kubernetes()
kub.pr()

{'apiVersion': 'v1',
 'kind': 'Pod',
 'metadata': {'name': 'gpu-pod-xmkqv', 'namespace': 'scb-usra'},
 'spec': {'containers': [{'args': ['pip install tqdm; sleep infinity'],
                          'command': ['/bin/bash', '-c'],
                          'image': 'gitlab-registry.nrp-nautilus.io/prp/jupyter-stack/prp:latest',
                          'name': 'gpu-container',
                          'resources': {'limits': {'nvidia.com/gpu': 1}},
                          'volumeMounts': [{'mountPath': '/scb-usra',
                                            'name': 'scb-usra'}]}],
          'restartPolicy': 'Never',
          'volumes': [{'name': 'scb-usra',
                       'persistentVolumeClaim': {'claimName': 'scb-usra'}}]}}


In [14]:
lo_ve(data= kub.kub_c, path= 'pod.yaml')

lo_ve: dumping dict_keys(['apiVersion', 'kind', 'metadata', 'spec']) 
 to pod.yaml


In [ ]:

	# apiVersion: v1
	# kind: Pod
	# metadata:
	# name: ata-pod
	# namespace: scb-usra
	# spec:
	# containers:
	# - name: scb-env-one
	# 	image: gitlab-registry.nrp-nautilus.io/usra/scb-env-one:latest
	# 	command: ["/bin/bash", "-c"]
	# 	args: [". activate base; sleep infinity"]
	# 	volumeMounts:
	# 	- mountPath: /scb-usra
	# 		name: scb-usra
	# 	resources:
	# 	limits:
	# 		memory: 10Gi
	# 		cpu: "1"
	# 	requests:
	# 		memory: 10Gi
	# 		cpu: "1"
	# volumes:
	# - name: scb-usra
	# 	persistentVolumeClaim:
	# 	claimName: scb-usra

	# kubectl create -f - << EOF
	# <contents you want to deploy>
	# EOF

	# kubectl get nodes -L nvidia.com/gpu.product

	# kubectl exec -it test-pod -- /bin/bash get into the pod
# 	`kubectl create -f cpu_pod.yaml` to create a pod

# `kubectl get pods -n scb-usra` to check the pod status

# `kubectl exec ata-pod -n scb-usra -it bash` to enter the pod

# `kubectl delete pod ata-pod -n scb-usra` to delete a pod
